In [10]:
# !pip install praw
# nltk.download()

In [1]:
import requests
import requests.auth
import pandas as pd
from newspaper import Article 
import praw
from praw.models import MoreComments
import json
import pyimgflip
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
import nltk
import re
from datetime import timedelta, datetime, date
import time

In [2]:
url = 'http://fox13now.com/2013/12/30/new-year-new-laws-obamacare-pot-guns-and-drones/'
article = Article(url)

In [3]:
with open('config.json') as f:
    data = json.load(f)
reddit_cred = data['Reddit']
watson_cred = data['Watson']
newspaper_cred = data['News']
sql_cred = data["SQL"]
img_cred = data["img"]

In [4]:
reddit = praw.Reddit(**reddit_cred)
reddit_cred

{'client_id': '_r9vZ43gHAboPQ',
 'client_secret': 'ryqXnZEMztMEBwTGXT1oSJHSIqA',
 'password': 'aFTeCTugiBEr',
 'user_agent': 'meme-times/0.1 by jaisal1024',
 'username': 'jaisal1024'}

In [5]:
conn_string = 'mysql://{user}:{password}@{host}/{db}?charset=utf8mb4'.format(
    host = sql_cred["host"], 
    user = sql_cred["user"],
    password = sql_cred["password"], 
    db = 'MemeNews')
engine = create_engine(conn_string)

In [6]:
comment_dict = { "post_id":[], 
                  'post_title':[],

                "id": [],
                        "author":[], 
                        "body":[],
                        "created": [],
                         'score':[],
                 'is_submitter':[],
                  'parent_id':[]}
articles_dict = { "title":[], 
                "score":[], 
                "id":[], "url":[], 
                "comms_num": [], 
                "created": [], 
                "body":[],
                "image":[],
                "keywords":[],
                "summary":[]
                  }
yest = (date.today()-timedelta(days = 1))
yest = datetime.combine(yest, datetime.min.time()).timestamp()
scrape_comments = 1

for submission in reddit.subreddit('news').hot(limit=10):
    if (submission.created > yest): 
        submission.comment_sort = 'best'
        article = Article(submission.url)
        try:
            article.download()
            article.parse()
            article.nlp()
            article.fetch_images()
        except:
            continue
        articles_dict["title"].append(re.sub(r'[^\x00-\x7F]', '', submission.title.replace('"', "'")))
        articles_dict["score"].append(submission.score)
        articles_dict["id"].append(submission.id)
        articles_dict["url"].append(submission.url)
        articles_dict["comms_num"].append(submission.num_comments)
        articles_dict["created"].append(submission.created)
        articles_dict["body"].append(re.sub(r'[^\x00-\x7F]', '',article.text.replace('"', "'")))
        articles_dict["image"].append(article.top_image)
        articles_dict["keywords"].append(', '.join(article.keywords).replace('"', "'"))
        articles_dict["summary"].append(re.sub(r'[^\x00-\x7F]', '', article.summary.replace('"', "'")))
        articles_data = pd.DataFrame(articles_dict)
        
        if (scrape_comments):
#             submission.comments.replace_more(limit=None)
            for top_level_comment in submission.comments.list()[:50]:
                try:
                    comment_dict['is_submitter'].append(top_level_comment.is_submitter)
                    comment_dict['post_id'].append(submission.id)
                    comment_dict['id'].append(top_level_comment.id)
                    comment_dict['author'].append(top_level_comment.author)
                    comment_dict['body'].append(re.sub(r'[^\x00-\x7F]', '', top_level_comment.body))
                    comment_dict['score'].append(top_level_comment.score)
                    comment_dict['created'].append(top_level_comment.created_utc)
                    comment_dict['parent_id'].append(top_level_comment.parent_id)
                    comment_dict['post_title'].append(submission.title)
                    comment_data = pd.DataFrame(comment_dict)
                except: 
                    continue            

In [7]:
articles_data

,body,comms_num,created,id,image,keywords,score,summary,title,url
0,Get the latest from TODAY Sign up for our news...,250,1.544561e+09,a56i3c,https://media4.s-nbcnews.com/j/newscms/2018_50...,"soe, khashoggi, today, lone, person, guardians...",2189,Maria Ressa is the editor of a Philippine news...,TIME's 2018 Person of the Year: 'The Guardians...,https://www.today.com/news/time-person-year-20...
1,"TALLAHASSEE, Fla. (AP) Florida officials say ...",2442,1.544523e+09,a52crr,https://apnews.com/images/ShareLogo.png,"state, received, ballots, totals, mailed, thou...",52627,"TALLAHASSEE, Fla. (AP) Florida officials say ...",Thousands of mailed ballots in Florida were no...,https://apnews.com/8488a401807045fcb642ea2cdf8...
2,We ever been to the moon? he asked.\n\nThe oth...,299,1.544552e+09,a55k6m,https://static01.nyt.com/images/2018/12/11/spo...,"warriors, mission, working, lunar, space, curr...",772,We ever been to the moon? he asked.\nTheyre go...,Stephen Curry Doubts Moon Landings. NASA Offer...,https://www.nytimes.com/2018/12/10/sports/step...
3,"'Students should see teachers as trustworthy, ...",131,1.544558e+09,a564or,https://WDRB.images.worldnow.com/images/179233...,"ban, kentucky, teachers, students, policies, s...",445,"'Students should see teachers as trustworthy, ...",Proposed legislation would ban paddling in Ken...,http://www.wdrb.com/story/39617740/proposed-le...
4,"In a nearly 5,000-year-old tomb in Sweden, res...",101,1.544548e+09,a5582d,https://cbsnews2.cbsistatic.com/hub/i/r/2018/1...,"neolithic, plague, spread, ago, researchers, 5...",684,"In a nearly 5,000-year-old tomb in Sweden, res...","Ancient, unknown strain of plague found in 5,0...",https://www.cbsnews.com/news/ancient-unknown-s...
5,"YouTube bans Proud Boys founder Gavin McInnes,...",9766,1.544497e+09,a4ygyx,https://s.hdnux.com/photos/77/33/15/16626496/3...,"tech, founder, proud, mcinnes, kick, giant, bo...",40521,"YouTube bans Proud Boys founder Gavin McInnes,...",Youtube Kicks off Proud Boys Founder Gavin Mci...,https://www.sfgate.com/entertainment/the-wrap/...
6,Committee Releases Report Revealing New Inform...,192,1.544495e+09,a4yalc,https://oversight.house.gov/wp-content/uploads...,"equifax, failed, consumers, report, gap, imple...",8195,Committee Releases Report Revealing New Inform...,House Oversight Committee releases report on e...,https://oversight.house.gov/report/committee-r...
7,LOS ANGELES Prosecutors say the former musica...,15,1.544558e+09,a565vu,https://cbsnews1.cbsistatic.com/hub/i/r/2018/1...,"music, feds, stole, los, face, kids, peace, fu...",215,LOS ANGELES Prosecutors say the former musica...,Prosecutors say the former musical director fo...,https://www.cbsnews.com/news/ex-un-music-chief...
8,Image copyright NASA/JPL Image caption The Voy...,1632,1.544486e+09,a4wt2s,https://ichef.bbci.co.uk/news/1024/branded_new...,"system, probes, probe, space, interstellar, ca...",34720,Image copyright NASA/JPL Image caption The Voy...,Voyager 2 leaves the Solar System,https://www.bbc.com/news/science-environment-4...
9,HONOLULU (HawaiiNewsNow) - With a smile on his...,29,1.544526e+09,a52t4n,http://www.hawaiinewsnow.com/resizer/R7HYIqcDe...,"sadao, man, 88yearold, 18, set, line, finishes...",912,HONOLULU (HawaiiNewsNow) - With a smile on his...,"After nearly 18 hours, 88-year-old man finishe...",http://www.hawaiinewsnow.com/2018/12/10/after-...


In [9]:
articles_data.to_sql('Daily_Articles', con = engine, if_exists='append', dtype={'None':VARCHAR(5)})

In [10]:
comment_data.to_sql('every_comment', con = engine, if_exists='append', dtype={'None':VARCHAR(5)})

In [8]:
comment_data

,author,body,created,id,is_submitter,parent_id,post_id,post_title,score
0,Dahhhkness,I'm really glad that the Khashoggi story didn'...,1.544537e+09,ebk9l85,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,675
1,jonovan,Why does the link go to Today.com and not Time...,1.544536e+09,ebk8bq3,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,159
2,NoahWebstersGhost,"""To stand up for truth is nothing. For truth, ...",1.544540e+09,ebkchcc,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,1
3,_lostintheroom,these people earned it,1.544534e+09,ebk6u66,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,180
4,JustAvgGuy,"Khashoggi made it it the cover, as he should.\...",1.544535e+09,ebk773f,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,135
5,DragonPup,There can be no democracy without a free press.,1.544537e+09,ebk92k5,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,69
6,ultrapaladin,I remember when TIME named Putin Person of the...,1.544539e+09,ebkbnti,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,1
7,noggin291,"While the recipients are certainly worthy, the...",1.544540e+09,ebkcsjn,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,1
8,CaptainGilliam,"> ""To critics on the right, Mueller is an over...",1.544538e+09,ebkalxb,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,1
9,Waldo_where_am_I,Murdering journalists is bad no exceptions. Bu...,1.544540e+09,ebkca8i,False,t3_a56i3c,a56i3c,TIME's 2018 Person of the Year: 'The Guardians...,1


In [11]:
def extractEntitiesFromUrl(url):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    
    watson_options = {
      "url": url,
      "features": {
        "entities": {
          "sentiment": True,
          "relevance": True,   
        }
      }
    }
    
    username = watson_cred["username"]
    password = watson_cred["password"]

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    data=resp.json()
    # create and return a dictionary for each entity with entity name, url, source, relevance and sentiment score as keys
    entities_list=[]
#     print(data)
    for entity in data["entities"]:
        entity_dict={}
        entity_dict["entity"]=entity["text"]
        entity_dict["relevance"]=entity["relevance"]
        entity_dict["sentiment"]=entity["sentiment"]["score"]
        entity_dict["entity"]=entity["text"]
        entities_list.append(entity_dict)
        
    return entities_list

def extractEntitiesFromComment(comment):
    endpoint_watson = "https://gateway.watsonplatform.net/natural-language-understanding/api/v1/analyze"
    params = {
        'version': '2017-02-27',
    }
    headers = { 
        'Content-Type': 'application/json',
    }
    
    watson_options = {
      "text": comment,
      "features": {
        "entities": {
          "sentiment": True,
          "relevance": True,
          "emotion" :True,
            'categories': True,
            'semantics': True
        }
      }
    }
    
    username = watson_cred["username"]
    password = watson_cred["password"]

    resp = requests.post(endpoint_watson, 
                         data=json.dumps(watson_options), 
                         headers=headers, 
                         params=params, 
                         auth=(username, password) 
                        )
    data=resp.json()
    # create and return a dictionary for each entity with entity name, url, source, relevance and sentiment score as keys
    entities_list=[]
    for entity in data["entities"]:
        entity_dict={}
        try: 
            entity_dict["emotion"]=entity["emotion"]
            entity_dict["entity"]=entity["text"]
            entity_dict["relevance"]=entity["relevance"]
            entity_dict["sentiment"]=entity["sentiment"]["score"]
            entities_list.append(entity_dict)
        except: 
            continue
        
    return entities_list

In [16]:
# entities= [extractEntitiesFromUrl(url) for url in df['url']]
entities = extractEntitiesFromUrl(df['url'][1])
# print(df['url'][1])
df_article_entities= pd.DataFrame(entities)
df_article_entities

,entity,relevance,sentiment
0,CHP,0.859324,-0.386687
1,Sacramento,0.460739,-0.503682
2,assault,0.453471,-0.473280
3,Porfirio Paz,0.419268,-0.341843
4,California,0.409612,-0.211332
5,Michael Williams,0.346818,0.163151
6,Guardian,0.332268,-0.541280
7,Facebook,0.244401,0.764963
8,Yvette Felarca,0.244176,0.000000
9,Ronald Cruz,0.243139,0.328550


In [12]:
meme_py = pyimgflip.Imgflip(username=img_cred["username"], password=img_cred["password"])

In [27]:
def generateMeme(num, raw): 
#     query = '''SELECT * FROM MemeNews.Daily_Article 
#     ORDER BY score, comms_num DESC LIMIT {0}'''.format(num)
    test_id = 'a4ygyx'
    query = '''SELECT * FROM MemeNews.Daily_Articles WHERE id LIKE '{0}' '''.format(test_id)
    df_articles = pd.read_sql(query, engine)
    j = 0
    for id_ in df_articles["id"]: 
        meme_dict = {"post_id": [], "meme_url": []}
        query = '''SELECT * FROM MemeNews.every_comment WHERE post_id LIKE '{0}' LIMIT 10'''.format(id_)
        df_comments =  pd.read_sql(query, engine)
        max_emotion_final = [('Joy', -1, df_comments["body"][0]),('Disgust', -1, df_comments["body"][0])]
        for body in df_comments["body"]: 
            if (len(body) > 200): 
                continue 
            resp = extractEntitiesFromComment(body)
            max_emotion = ('emotion', -1, '', 0)
            for r in resp: 
                for key, value in r['emotion'].items():
                    print(key, value)
                    if value > max_emotion[1]: 
                        max_emotion = (key, value, body, r['sentiment'])         
            if max_emotion[3] > 0: #sentiment is positive
                if (max_emotion[1] > max_emotion_final[0][1]):
                    max_emotion_final[0] = max_emotion
            elif max_emotion[1] > max_emotion_final[1][1]: #sentiment is negative
                max_emotion_final[1] = max_emotion
        
        if (max_emotion_final[0] == max_emotion_final[1]): #same emotion   
            query = '''SELECT * FROM MemeNews.Meme_Photos where emotion LIKE '{0}' ORDER BY RAND() LIMIT 2'''.format(max_emotion_1[0])
            df_memes_photos = pd.read_sql(query, engine)
            i = 0
            for meme_photo in df_memes_photos: 
                result = meme_py.caption_image(meme_photo, max_emotion_final[i][2], "")
                meme_dict["post_id"].append(id_)
                meme_dict["meme_url"].append(result["url"])
                i+=1
        else: 
            i = 0
            for max_emotion in max_emotion_final: 
                query = '''SELECT * FROM MemeNews.Meme_Photos where emotion LIKE '{0}' ORDER BY RAND() LIMIT 1'''.format(max_emotion_final[i][0])
                df_memes_photos = pd.read_sql(query, engine)
                result = meme_py.caption_image(df_memes_photos["id"], max_emotion_final[i][2], "")
                meme_dict["post_id"].append(id_)
                meme_dict["meme_url"].append(result["url"])
                i+=1
        

        print(meme_dict)
        df = pd.DataFrame(meme_dict, (j, j+1))
        j+=2

    return df




In [28]:
meme_ = generateMeme(0, True)
meme_

sadness 0.451651
joy 0.353964
fear 0.048503
disgust 0.047534
anger 0.095738
{'post_id': ['a4ygyx', 'a4ygyx'], 'meme_url': ['http://i.imgflip.com/2opd39.jpg', 'http://i.imgflip.com/2opd3c.jpg']}
sadness 0.451651
joy 0.353964
fear 0.048503
disgust 0.047534
anger 0.095738
{'post_id': ['a4ygyx', 'a4ygyx'], 'meme_url': ['http://i.imgflip.com/2opd3g.jpg', 'http://i.imgflip.com/2opd3l.jpg']}


,meme_url,post_id
2,http://i.imgflip.com/2opd3g.jpg,a4ygyx
3,http://i.imgflip.com/2opd3l.jpg,a4ygyx


In [32]:
meme_["sentiment"] = ['negative', 'positive']
meme_

,meme_url,post_id,sentiment
2,http://i.imgflip.com/2opd3g.jpg,a4ygyx,negative
3,http://i.imgflip.com/2opd3l.jpg,a4ygyx,positive


In [33]:
meme_.to_sql('Memes', con = engine, if_exists='replace', dtype={'None':VARCHAR(5)})

In [34]:
query = '''SELECT * FROM MemeNews.Memes'''
df_memes_ = pd.read_sql(query, engine)
df_memes_

,index,meme_url,post_id,sentiment
0,2,http://i.imgflip.com/2opd3g.jpg,a4ygyx,negative
1,3,http://i.imgflip.com/2opd3l.jpg,a4ygyx,positive


In [40]:
for index, row in df_memes_.iterrows():
    if (index %2 ==0): 
        query = '''SELECT * FROM MemeNews.Daily_Articles WHERE id LIKE '{0}' LIMIT 1'''.format(row['post_id'])
        df_article = pd.read_sql(query, engine)
        print(df_article['title'], df_article['image'], df_article['keywords'], df_article['url'], df_article['summary'])

0    Youtube Kicks off Proud Boys Founder Gavin Mci...
Name: title, dtype: object 0    https://s.hdnux.com/photos/77/33/15/16626496/3...
Name: image, dtype: object 0    giant, rally, account, mcinnes, kick, tech, pr...
Name: keywords, dtype: object 0    https://www.sfgate.com/entertainment/the-wrap/...
Name: url, dtype: object 0    YouTube bans Proud Boys founder Gavin McInnes,...
Name: summary, dtype: object
